In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
# Re-import necessary libraries due to execution state reset
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from numba import njit

In [15]:


# Function to generate correlated Brownian motions
@njit
def generate_correlated_BMs(N, M, rho):
    Z1 = np.random.randn(N, M)
    Z2 = np.random.randn(N, M)
    W1 = Z1
    W2 = rho * Z1 + np.sqrt(1 - rho ** 2) * Z2
    return W1, W2

# Function to simulate stock price and variance paths using Euler-Maruyama
@njit
def simulate_heston_paths(S0, v0, kappa, theta, sigma, r, rho, T, M, N):
    dt = T / M
    S = np.zeros((N, M + 1))
    v = np.zeros((N, M + 1))
    S[:, 0] = S0
    v[:, 0] = v0

    W1, W2 = generate_correlated_BMs(N, M, rho)

    for t in range(1, M + 1):
        v[:, t] = np.maximum(v[:, t-1] + kappa * (theta - v[:, t-1]) * dt + sigma * np.sqrt(v[:, t-1] * dt) * W2[:, t-1], 0)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * v[:, t-1]) * dt + np.sqrt(v[:, t-1] * dt) * W1[:, t-1])

    return S, v

# Function for backward induction using least squares regression
def least_squares_monte_carlo(S, K, r, T, M):
    dt = T / M
    intrinsic_values = np.maximum(K - S, 0)
    cashflows = intrinsic_values[:, -1].copy()

    for t in range(M - 1, 0, -1):
        in_the_money = intrinsic_values[:, t] > 0
        X = S[in_the_money, t].reshape(-1, 1)
        Y = cashflows[in_the_money] * np.exp(-r * dt)

        if len(Y) > 0:
            model = LinearRegression().fit(X, Y)
            continuation_value = model.predict(X)
            exercise = intrinsic_values[in_the_money, t] > continuation_value
            cashflows[in_the_money] = np.where(exercise, intrinsic_values[in_the_money, t], cashflows[in_the_money] * np.exp(-r * dt))

    return np.mean(cashflows) * np.exp(-r * dt)

# Function to run multiple simulations
def run_simulations(S0, K, T, r, kappa, theta, sigma, rho, v0, M, N, num_simulations):
    option_prices = np.zeros(num_simulations)

    for i in range(num_simulations):
        S, v = simulate_heston_paths(S0, v0, kappa, theta, sigma, r, rho, T, M, N)
        option_prices[i] = least_squares_monte_carlo(S, K, r, T, M)

    return option_prices

# Set parameters
np.random.seed(0)  # For reproducibility
S0 = 100    # Initial stock price
K = 100     # Strike price
T = 1       # Maturity (1 year)
r = 0.05    # Risk-free rate
kappa = 2.0 # Mean reversion speed of variance
theta = 0.04 # Long-run variance
sigma = 0.2 # Volatility of variance (vol of vol)
rho = -0.7  # Correlation between stock and variance
v0 = 0.04   # Initial variance
M = 50      # Time steps
N = 10000   # Number of simulations
num_simulations = 100  # Number of runs

# Run the vectorized simulation
option_prices = run_simulations(S0, K, T, r, kappa, theta, sigma, rho, v0, M, N, num_simulations)

# Display results
df_results = pd.DataFrame(option_prices, columns=["Option Price"])
df_results


,Option Price
0,5.926042
1,6.066972
2,6.002040
3,6.041851
4,6.042264
...,...
95,6.113136
96,6.163203
97,5.938309
98,5.954327
